In [1236]:
import numpy as np
from PIL import Image,ImageOps

In [1245]:
def Barrycenter(cost,a, b, t, error=0.0000001,regular=0.000000001):
    #a b images, more t more b, less t more a, error of regularised result, regular degree of regularisation
    K = np.exp(-cost/regular)
    #K is regularisation step
    
    va=np.ones(n)/n
    vb=np.ones(n)/n
    alpha = np.ones(n)/n
    #Zwischenschritte, after Sinkhorn
    
    oldalpha = 5
    #saves old value bc result alpha should converge, if difference > error continue, < then continue
    while np.linalg.norm(alpha-oldalpha)>error:
        oldalpha =alpha
        
        ua = np.maximum(np.dot(K, va),np.finfo(np.float).eps)
        ub = np.maximum(np.dot(K, vb),np.finfo(np.float).eps)
        
        ua = np.true_divide(a,ua)
        ub = np.true_divide(b,ub)
        
        alpha = np.power(np.dot(np.transpose(K), ua),1-t)
        alpha = alpha * np.power(np.dot(np.transpose(K), ub),t)
        
        va = np.maximum(np.dot(np.transpose(K), ua),np.finfo(np.float).eps)
        vb = np.maximum(np.dot(np.transpose(K), ub),np.finfo(np.float).eps)
        
        va = np.true_divide(alpha,va)
        vb = np.true_divide(alpha,vb)


    return alpha

In [1246]:
#cost functions/metrics
l2 = lambda x,y: np.sqrt(x*x+y*y)
l1 = lambda x,y: np.abs(x)+np.abs(y)

In [1250]:
#closer to a or to be
t=0.6

str1 = "MINIlegBOcrop.PNG"
str2 = "MINIlegBOcropHEU.PNG"

#norm, can be replaced by other cost function
norm = l1

#image put in array, shape flattened, inverted and normalised such that they sum to 1
a=np.asarray(ImageOps.grayscale(Image.open(str1)))
b=np.asarray(ImageOps.grayscale(Image.open(str2)))

x,y = a.shape

a=a.flatten()
b=b.flatten()
a=255-a
b=255-b
a = a/np.sum(a)
b = b/np.sum(b)
a = a.astype(np.longdouble) 
b = b.astype(np.longdouble) 

#initialising of cost matrix
n=x*y
cost = np.array([[norm(i//x-j//x , i%y-j%y) for i in range(n)] for j in range(n)])
cost = cost.astype(np.longdouble)  
#normalising 
cost = cost/np.sum(cost)
 
#Barrycenter called
barry = Barrycenter(cost,a,b,t)
barry = barry/np.max(barry)
barry = (barry*255).astype(np.uint8)
barry = 255-barry
barry = barry.reshape((x,y))                

In [1251]:
#array turned to image
img = Image.fromarray(barry, "L")
img.show()
#img.save("NumpyPrH.PNG")